# Load packages

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.evalModel import evalLogisticModels, getPredDist

# Data

## Load cell lines

In [3]:
trainRNA = pd.read_csv('../../data/processed/RNA_train_cancergenes.csv', index_col=0)
trainCellLines = list(trainRNA.index)

testRNA = pd.read_csv('../../data/processed/RNA_test_cancergenes.csv', index_col=0)
testCellLines = list(testRNA.index)

newRNA = pd.read_csv('../../data/processed/RNA_newcancer_cancergenes.csv', index_col=0)
newCellLines = list(newRNA.index)

## CDR

In [4]:
cdr = pd.read_csv('../../data/processed/drugCellLinePairsData.csv', index_col='DepMap_ID')
trainCDR = cdr.loc[trainCellLines, :].reset_index()
testCDR = cdr.loc[testCellLines, :].reset_index()
newCDR = cdr.loc[newCellLines, :].reset_index()

In [5]:
testTemp = testCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                             'name': 'drug',
                                                                                             'effective': 'true'})

newTemp = newCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                          'name': 'drug',
                                                                                          'effective': 'true'})

# Load drugs

In [6]:
drugs = pd.read_csv('../../data/processed/drug_fingerprints.csv', index_col=0)

drugs.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
cytarabine,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
epinastine,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
floxuridine,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
valrubicin,1,0,0,0,0,1,0,0,1,0,...,0,1,0,1,1,1,0,1,0,0
adapalene,1,1,1,0,0,1,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0


In [7]:
trainDrugs = drugs.loc[list(trainCDR.name.values), :].to_numpy()
testDrugs = drugs.loc[list(testCDR.name.values), :].to_numpy()
newDrugs = drugs.loc[list(newCDR.name.values), :].to_numpy()

trainRNA = trainRNA.loc[list(trainCDR.DepMap_ID.values), :].to_numpy()
testRNA = testRNA.loc[list(testCDR.DepMap_ID.values), :].to_numpy()
newRNA = newRNA.loc[list(newCDR.DepMap_ID.values), :].to_numpy()

In [8]:
trainData = [trainDrugs, trainRNA]
trainEff = trainCDR.effective.to_numpy()
testData = [testDrugs, testRNA]
newData = [newDrugs, newRNA]

In [9]:
del cdr, drugs, trainDrugs, testDrugs, newDrugs, trainRNA, testRNA, newRNA

# Fusion performance

In [10]:
drugPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/drugs/siameseV1/models/'
drugModel = 'DrugFewShot_Layers1_Hidden64_DO0-1_AFrelu_LR0-001_DR0-99_DS1000'
drugPath = os.path.join(drugPath, drugModel)

In [11]:
rnaPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/cellLines/siameseV1/models/'
rnaModel = 'CellLineFewShot_Layers2_Hidden64_DO0-1_AFrelu_LR0-001_DR0-99_DS1000'
rnaPath = os.path.join(rnaPath, rnaModel)

In [12]:
fusionPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/fusion/embedDrug_embedRNA_V2/models'
bestModel = 'FusionFewShotEmbedDrugEmbedCell_NL128_64_DO0-1_AFrelu_LR0-001_DR0-99_DS1024_BYrna'
fusionPath = os.path.join(fusionPath, bestModel)

In [13]:
evalLog = evalLogisticModels(trainData.copy(), trainEff, testData.copy(), newData.copy(), alt='xgb',
                            fusionPath=fusionPath, drugPath=drugPath, rnaPath=rnaPath)

In [23]:
testDF, newDF, testWrong, newWrong, testCounts, newCounts = evalLog.evaluate(testDF=testTemp.copy(), 
                                                                            newDF=newTemp.copy(),
                                                                            thresh=0.1)


Average Cell Line precision @ k on test set
	Precision@1: 0.7647
	Precision@2: 0.8039
	Precision@3: 0.8105
	Precision@4: 0.8186
	Precision@5: 0.8314
	Precision@10: 0.8179

Average Cell Line precision @ k on newcancer set
	Precision@1: 0.8308
	Precision@2: 0.8615
	Precision@3: 0.8462
	Precision@4: 0.8269
	Precision@5: 0.8154
	Precision@10: 0.8098


Test set:

	# of cell lines without effective drug among top-3 recs: 0
	# of unique drugs among top-3 predictions: 17

New cancer set
No true effective drugs identified in top 3 for ACH-000268 (top drug: sangivamycin)

	# of cell lines without effective drug among top-3 recs: 1
	# of unique drugs among top-3 predictions: 20


In [15]:
testDist = getPredDist(testDF)

Avg varaince of predictions for each drug: 0.0015999999595806003


In [16]:
newDist = getPredDist(newDF)

Avg varaince of predictions for each drug: 0.0017999999690800905


In [17]:
testCounts.head()

,1,2,3,total
echinomycin,14,4,5,23
dolastatin-10,9,8,6,23
epothilone-b,6,6,11,23
nemorubicin,5,4,2,11
genz-644282,5,3,3,11


## Cancer precision

In [24]:
cancerTest, cancerNew = evalLog.getCancerPerformance(testDF.copy(), newDF.copy())

In [25]:
print(round(cancerTest.mean(), 4))
cancerTest

p1    0.7766
p2    0.8090
p3    0.8085
p4    0.8220
p5    0.8336
dtype: float64


,p1,p2,p3,p4,p5
Liver Cancer,1.000000,1.000000,1.000000,1.000000,1.000000
Bladder Cancer,1.000000,1.000000,0.888889,0.916667,0.933333
Brain Cancer,1.000000,0.875000,0.833333,0.750000,0.800000
Head and Neck Cancer,1.000000,0.833333,0.888889,0.833333,0.800000
Endometrial/Uterine Cancer,1.000000,0.833333,0.777778,0.833333,0.800000
Skin Cancer,0.800000,0.900000,0.800000,0.850000,0.840000
Lung Cancer,0.769231,0.807692,0.846154,0.826923,0.846154
Ovarian Cancer,0.750000,0.875000,0.833333,0.875000,0.850000
Esophageal Cancer,0.666667,0.666667,0.666667,0.750000,0.800000
Colon/Colorectal Cancer,0.500000,0.750000,0.833333,0.875000,0.900000


In [22]:
print(round(cancerNew.mean(), 4))
cancerNew

p1    0.8911
p2    0.8918
p3    0.8788
p4    0.8323
p5    0.8205
dtype: float64


,p1,p2,p3,p4,p5
Prostate Cancer,1.000000,1.000000,1.000000,1.000000,0.900000
Gallbladder Cancer,1.000000,1.000000,1.000000,0.750000,0.800000
Neuroblastoma,1.000000,1.000000,0.888889,0.833333,0.800000
Sarcoma,1.000000,0.916667,0.888889,0.875000,0.866667
Rhabdoid,1.000000,0.875000,0.916667,0.812500,0.850000
Bone Cancer,0.888889,0.888889,0.925926,0.916667,0.888889
Gastric Cancer,0.857143,0.892857,0.857143,0.892857,0.871429
Bile Duct Cancer,0.800000,0.700000,0.733333,0.750000,0.720000
Thyroid Cancer,0.750000,0.875000,0.833333,0.781250,0.800000
Kidney Cancer,0.615385,0.769231,0.743590,0.711538,0.707692
